# **Co-occurrence Matrix**
BoW é muito simples e fácil de entender, mas tem algumas limitações. Normalmente ele ignora a correlação entre as palavras, o que deixa a semântica mais fraca. A abordagem por co occurrence tem como objetivo analisar a vizinhança das palavras, para a partir dai capturar mais semântica no texto. As linhas e as colunas são palavras do vocabulário, e cada célula vai armazenar a quantidade de vezes que duas palavras ocorreram.

### **Example**

In [1]:
C = ['The who is the band!', 'who is the band?', 'The band who plays the who.']

print('C has %d texts:' % len(C))
for i in range(len(C)):
    print(f"t{i+1} = {C[i]}")

C has 3 texts:
t1 = The who is the band!
t2 = who is the band?
t3 = The band who plays the who.


In [2]:
import re

def pre_process_corpus(corpus):
    new_corpus = [doc.lower() for doc in corpus]
    regex = r"(?<!\d)[\!\?.,;:-](?!\d)"
    return [re.sub(regex, "", doc, 0) for doc in new_corpus]

In [3]:
import sklearn
import pandas as pd
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer

corpus = pre_process_corpus(C)
print(corpus)

['the who is the band', 'who is the band', 'the band who plays the who']


In [7]:
vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names_out()

print(pd.DataFrame(doc_term_matrix.A, columns=terms).to_string())

   band  is  plays  the  who
0     1   1      0    2    1
1     1   1      0    1    1
2     1   0      1    2    2


In [5]:
vectorizer.vocabulary_

{'the': 3, 'who': 4, 'is': 1, 'band': 0, 'plays': 2}

In [11]:
co_occurrence_matrix = (doc_term_matrix.T * doc_term_matrix)
print(co_occurrence_matrix.todense())

[[3 2 1 5 4]
 [2 2 0 3 2]
 [1 0 1 2 2]
 [5 3 2 9 7]
 [4 2 2 7 6]]


Para cada palavra, eu sei quais são os seus vizinhos mais frequentes em cada coleção.

In [12]:
import scipy.sparse as sp
g = sp.diags(1./co_occurrence_matrix.diagonal())
co_occurrence_matrix_norm = g * co_occurrence_matrix # Normalized matrix

print(co_occurrence_matrix_norm.todense())

[[1.         0.66666667 0.33333333 1.66666667 1.33333333]
 [1.         1.         0.         1.5        1.        ]
 [1.         0.         1.         2.         2.        ]
 [0.55555556 0.33333333 0.22222222 1.         0.77777778]
 [0.66666667 0.33333333 0.33333333 1.16666667 1.        ]]


Normalizar os valores da matriz ficando em uma escala menor.

In [13]:
co_occurrence_matrix.setdiag(0)
print(co_occurrence_matrix.todense())

[[0 2 1 5 4]
 [2 0 0 3 2]
 [1 0 0 2 2]
 [5 3 2 0 7]
 [4 2 2 7 0]]


Zerar a diagonal principal, afinal não tem pq eu querer comparar uma palavra com ela mesma.

In [14]:
import numpy as np
from collections import defaultdict

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    
    for text in sentences:
        text = text.lower().split() # preprocessing, use tokenizer instead
        
        for i in range(len(text)):
            token = text[i]
            vocab.add(token) # add to vocab
            next_token = text[i + 1: i + 1 + window_size]
            
            for t in next_token:
                key = tuple(sorted([t, token]))
                d[key] += 1
    
    # formulate the dictionary into dataframe
    vocab = sorted(vocab)
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16), index=vocab, columns=vocab)
    
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
        
    return df

In [16]:
df = co_occurrence(corpus, 3)
df

,band,is,plays,the,who
band,0,2,1,4,3
is,2,0,0,3,2
plays,1,0,0,2,2
the,4,3,2,1,6
who,3,2,2,6,1


Posso alterar o tamanho da janela para saber a qual distância ocorreu cada palavra. Quantas vezes, uma palavra apareceu distância da outra em uma distância window_size.